In [29]:
# Data Loading

from importlib import reload
from model import dataset

reload(dataset)

# Define the configuration for the Data Loader
LOADER_CONFIG = {
    'batch_size': 16,
    'use_data_cache': True,
    'img_dimension': 32,
}

# Define the transformation
transform 

# Create the Data Loader
train_loader, test_loader, label_map = dataset.load(LOADER_CONFIG)
num_classes = sum(label is not None for label in label_map)

print(f"Num Train Samples: {len(train_loader)}")
print(f"Num Test Samples: {len(test_loader)}")
print(f"Num Labels: {sum(label is not None for label in label_map)}")

Num Train Samples: 1077
Num Test Samples: 264
Num Labels: 11


In [28]:
# Model Loading

from importlib import reload
from model import bsnn

reload(bsnn)

BSNN_CONFIG = {
    'num_blocks': 2,
    'num_classes': num_classes,
    'img_dimension': 32,
}

model = bsnn.BSNN(config=BSNN_CONFIG)
print(model)

BSNN(
  (layers): Sequential(
    (conv0): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
    (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (relu0): Leaky()
    (conv1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (relu1): Leaky()
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (output): Linear(in_features=400, out_features=11, bias=True)
    (dropout): Dropout(p=0.25, inplace=False)
    (leaky_out): Leaky()
  )
)


In [ ]:
# Training

from importlib import reload
from model import train

reload(train)

TRAIN_CONFIG = {
    'num_steps': 15,
    'num_epochs': 50,
    'learning_rate': 0.0001,
    'thresholds': [2, 2, 3],
    'activation_threshold': 0.4,
}

assert(len(TRAIN_CONFIG['thresholds']) == BSNN_CONFIG['num_blocks'])

